In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark= SparkSession. \
builder. \
appName("week_7_assignment_one"). \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/itv009033/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
spark

# 1. You are working as a Data Engineer in ABC Company. You are dealing with a large dataset of customer transactions (/public/trendytech/datasets/cust_transf.csv) in HDFS, including information such as customer ID, purchase date, product id, and amount.

# A) Design a caching mechanism using dataframes to enhance the performance of data retrieval for the following use cases

<h3>A.1 Your marketing team wants to identify the top-selling products based on
revenue for a given time period. The query is expected to be executed
frequently, and the results need to be returned quickly. Design a caching
strategy that efficiently retrieves the top-selling products by revenue.
Additionally, demonstrate the impact of caching by comparing the retrieval
time for Top 10 best-selling products from start_date = "2023-05-01" to
end_date = "2023-06-08" before and after implementing the caching strategy.
[Note : Strategize your caching in such a way that the right Dataframes are
cached at the right time for maximal performance gains]</h3>

<h8>Without caching : </h8>

In [3]:
# customer ID, purchase date, product id, and amount.

customer_schema = "customer_ID long, purchase_date date, product_id long,amount float"

In [4]:
customers_df = spark.read.format("csv").schema(customer_schema).load("/public/trendytech/datasets/cust_transf.csv")
customers_df.show()

+-----------+-------------+----------+------+
|customer_ID|purchase_date|product_id|amount|
+-----------+-------------+----------+------+
|       1001|   2023-05-15|      1001| 49.99|
|       1002|   2023-05-16|      1002| 29.99|
|       1003|   2023-05-17|      1003| 39.99|
|       1004|   2023-05-18|      1004| 19.99|
|       1005|   2023-05-19|      1005| 24.99|
|       1001|   2023-05-20|      1002| 29.99|
|       1002|   2023-05-21|      1003| 39.99|
|       1003|   2023-05-22|      1004| 19.99|
|       1004|   2023-05-23|      1005| 24.99|
|       1005|   2023-05-24|      1001| 49.99|
|       1001|   2023-05-25|      1003| 39.99|
|       1002|   2023-05-26|      1004| 19.99|
|       1003|   2023-05-27|      1005| 24.99|
|       1004|   2023-05-28|      1001| 49.99|
|       1005|   2023-05-29|      1002| 29.99|
|       1001|   2023-05-30|      1003| 39.99|
|       1002|   2023-05-31|      1004| 19.99|
|       1003|   2023-06-01|      1005| 24.99|
|       1004|   2023-06-02|      1

In [5]:
filtered = customers_df.filter((customers_df.purchase_date>="2023-05-01") & (customers_df.purchase_date<="2023-06-08"))

In [6]:
filtered.groupBy("product_id").sum("amount").withColumnRenamed("sum(amount)", "Revenue").show()

+----------+--------------------+
|product_id|             Revenue|
+----------+--------------------+
|      1010|   7312.910350799561|
|      1002| 4.293836211229706E8|
|      1012|    9402.91035079956|
|      1009|   6267.909952163696|
|      1013|   10447.91035079956|
|      1007|   4177.909952163696|
|      1011|    8357.91035079956|
|      1005|2.7828563865119934E8|
|      1001| 5.566826598912048E8|
|      1015|   12537.91035079956|
|      1014|   11492.91035079956|
|      1008|   5222.909952163696|
|      1004| 2.862080211229706E8|
|      1006|  3132.9099521636963|
|      1003| 5.725592484315491E8|
+----------+--------------------+



<h8>With caching : </h8>

In [7]:
customers_cached = customers_df.cache()

In [8]:
customers_cached.show()

+-----------+-------------+----------+------+
|customer_ID|purchase_date|product_id|amount|
+-----------+-------------+----------+------+
|       1001|   2023-05-15|      1001| 49.99|
|       1002|   2023-05-16|      1002| 29.99|
|       1003|   2023-05-17|      1003| 39.99|
|       1004|   2023-05-18|      1004| 19.99|
|       1005|   2023-05-19|      1005| 24.99|
|       1001|   2023-05-20|      1002| 29.99|
|       1002|   2023-05-21|      1003| 39.99|
|       1003|   2023-05-22|      1004| 19.99|
|       1004|   2023-05-23|      1005| 24.99|
|       1005|   2023-05-24|      1001| 49.99|
|       1001|   2023-05-25|      1003| 39.99|
|       1002|   2023-05-26|      1004| 19.99|
|       1003|   2023-05-27|      1005| 24.99|
|       1004|   2023-05-28|      1001| 49.99|
|       1005|   2023-05-29|      1002| 29.99|
|       1001|   2023-05-30|      1003| 39.99|
|       1002|   2023-05-31|      1004| 19.99|
|       1003|   2023-06-01|      1005| 24.99|
|       1004|   2023-06-02|      1

In [9]:
customers_cached.count()

87498290

In [10]:
customers_cached.count()

87498290

In [11]:
filtered_cache = customers_cached.filter((customers_cached.purchase_date >= "2023-05-01") & (customers_cached.purchase_date<="2023-06-08")).cache()

In [12]:
filtered_cache.show()

+-----------+-------------+----------+------+
|customer_ID|purchase_date|product_id|amount|
+-----------+-------------+----------+------+
|       1001|   2023-05-15|      1001| 49.99|
|       1002|   2023-05-16|      1002| 29.99|
|       1003|   2023-05-17|      1003| 39.99|
|       1004|   2023-05-18|      1004| 19.99|
|       1005|   2023-05-19|      1005| 24.99|
|       1001|   2023-05-20|      1002| 29.99|
|       1002|   2023-05-21|      1003| 39.99|
|       1003|   2023-05-22|      1004| 19.99|
|       1004|   2023-05-23|      1005| 24.99|
|       1005|   2023-05-24|      1001| 49.99|
|       1001|   2023-05-25|      1003| 39.99|
|       1002|   2023-05-26|      1004| 19.99|
|       1003|   2023-05-27|      1005| 24.99|
|       1004|   2023-05-28|      1001| 49.99|
|       1005|   2023-05-29|      1002| 29.99|
|       1001|   2023-05-30|      1003| 39.99|
|       1002|   2023-05-31|      1004| 19.99|
|       1003|   2023-06-01|      1005| 24.99|
|       1004|   2023-06-02|      1

In [13]:
filtered_cache.count()

65226530

In [14]:
filtered_cache.count()

65226530

In [15]:
filtered_cache.groupBy("product_id").sum("amount").withColumnRenamed("sum(amount)", "Revenue").show()

+----------+--------------------+
|product_id|             Revenue|
+----------+--------------------+
|      1010|   7312.910350799561|
|      1002| 4.293836211229706E8|
|      1012|    9402.91035079956|
|      1009|   6267.909952163696|
|      1013|   10447.91035079956|
|      1007|   4177.909952163696|
|      1011|    8357.91035079956|
|      1005|2.7828563865119934E8|
|      1001| 5.566826598912048E8|
|      1015|   12537.91035079956|
|      1014|   11492.91035079956|
|      1008|   5222.909952163696|
|      1004| 2.862080211229706E8|
|      1006|  3132.9099521636963|
|      1003| 5.725592484315491E8|
+----------+--------------------+



# A.2 Find the top 10 customers with maximum transaction amount for the same date range of start_date = "2023-05-01" to end_date = "2023-06-08"

In [16]:
filtered_cache.groupBy("customer_ID").max("amount").withColumnRenamed("max(amount)", "maximum_amount").sort("maximum_amount", ascending = False).limit(10).show()

+-----------+--------------+
|customer_ID|maximum_amount|
+-----------+--------------+
|       1015|         59.99|
|       1014|         54.99|
|       1011|         49.99|
|       1013|         49.99|
|       1006|         49.99|
|       1005|         49.99|
|       1004|         49.99|
|       1001|         49.99|
|       1012|         44.99|
|       1007|         39.99|
+-----------+--------------+



# A.3 Implement all of the above using Spark Table (Create an External Table).

In [17]:
spark.sql("create database if not exists it009033_customers")

""


In [18]:
spark.sql("show databases")

databaseName
00000assign7_itv0...
0001_av_ivy_tesco
001_retail
003402_hive1
005198_ivy_tesco
005212_ivy_tesco
005222_ivy_practice
005260_ivy_database
005876_week5_assi...
005933_retail


In [19]:
spark.sql("use it009033_customers")

""


In [20]:
spark.sql("create table if not exists it009033_customers.customers_external(customer_ID long, purchase_date date, product_id long,amount float) using csv location '/public/trendytech/datasets/cust_transf.csv'")

""


In [21]:
spark.sql("show tables").show()

+------------------+------------------+-----------+
|          database|         tableName|isTemporary|
+------------------+------------------+-----------+
|it009033_customers|customers_external|      false|
+------------------+------------------+-----------+



In [22]:
spark.sql("select * from customers_external").show()

+-----------+-------------+----------+------+
|customer_ID|purchase_date|product_id|amount|
+-----------+-------------+----------+------+
|       1001|   2023-05-15|      1001| 49.99|
|       1002|   2023-05-16|      1002| 29.99|
|       1003|   2023-05-17|      1003| 39.99|
|       1004|   2023-05-18|      1004| 19.99|
|       1005|   2023-05-19|      1005| 24.99|
|       1001|   2023-05-20|      1002| 29.99|
|       1002|   2023-05-21|      1003| 39.99|
|       1003|   2023-05-22|      1004| 19.99|
|       1004|   2023-05-23|      1005| 24.99|
|       1005|   2023-05-24|      1001| 49.99|
|       1001|   2023-05-25|      1003| 39.99|
|       1002|   2023-05-26|      1004| 19.99|
|       1003|   2023-05-27|      1005| 24.99|
|       1004|   2023-05-28|      1001| 49.99|
|       1005|   2023-05-29|      1002| 29.99|
|       1001|   2023-05-30|      1003| 39.99|
|       1002|   2023-05-31|      1004| 19.99|
|       1003|   2023-06-01|      1005| 24.99|
|       1004|   2023-06-02|      1

In [23]:
spark.sql("select product_id, sum(amount) as revenue from it009033_customers.customers_external where purchase_date>= '2023-05-01' and purchase_date <= '2023-06-30' group by 1 order by sum(amount) desc limit 10").show()

+----------+-------------------+
|product_id|            revenue|
+----------+-------------------+
|      1001|8.747870369718933E8|
|      1003|6.997946369718933E8|
|      1002|5.248022035947418E8|
|      1005|4.373060035947418E8|
|      1004|3.498098035947418E8|
|      1015|  12537.91035079956|
|      1014|  11492.91035079956|
|      1013|  10447.91035079956|
|      1012|   9402.91035079956|
|      1011|   8357.91035079956|
+----------+-------------------+



In [24]:
spark.sql("select customer_ID, max(amount) as maximum_amount from it009033_customers.customers_external where purchase_date>= '2023-05-01' and purchase_date <= '2023-06-30' group by 1 order by max(amount) desc limit 10").show()

+-----------+--------------+
|customer_ID|maximum_amount|
+-----------+--------------+
|       1015|         59.99|
|       1014|         54.99|
|       1009|         49.99|
|       1001|         49.99|
|       1013|         49.99|
|       1004|         49.99|
|       1005|         49.99|
|       1006|         49.99|
|       1011|         49.99|
|       1010|         49.99|
+-----------+--------------+



In [25]:
spark.sql("cache lazy table it009033_customers.customers_external")

""


In [26]:
spark.sql("select product_id, sum(amount) as revenue from it009033_customers.customers_external where purchase_date>= '2023-05-01' and purchase_date <= '2023-06-30' group by 1 order by sum(amount) desc limit 10").show()

+----------+-------------------+
|product_id|            revenue|
+----------+-------------------+
|      1001|8.747870369718933E8|
|      1003|6.997946369718933E8|
|      1002|5.248022035947418E8|
|      1005|4.373060035947418E8|
|      1004|3.498098035947418E8|
|      1015|  12537.91035079956|
|      1014|  11492.91035079956|
|      1013|  10447.91035079956|
|      1012|   9402.91035079956|
|      1011|   8357.91035079956|
+----------+-------------------+



In [27]:
spark.sql("select product_id, sum(amount) as revenue from it009033_customers.customers_external where purchase_date>= '2023-05-01' and purchase_date <= '2023-06-30' group by 1 order by sum(amount) desc limit 10").show()

+----------+-------------------+
|product_id|            revenue|
+----------+-------------------+
|      1001|8.747870369718933E8|
|      1003|6.997946369718933E8|
|      1002|5.248022035947418E8|
|      1005|4.373060035947418E8|
|      1004|3.498098035947418E8|
|      1015|  12537.91035079956|
|      1014|  11492.91035079956|
|      1013|  10447.91035079956|
|      1012|   9402.91035079956|
|      1011|   8357.91035079956|
+----------+-------------------+



In [28]:
spark.sql("select customer_ID, max(amount) as maximum_amount from it009033_customers.customers_external where purchase_date>= '2023-05-01' and purchase_date <= '2023-06-30' group by 1 order by max(amount) desc limit 10").show()

+-----------+--------------+
|customer_ID|maximum_amount|
+-----------+--------------+
|       1015|         59.99|
|       1014|         54.99|
|       1005|         49.99|
|       1006|         49.99|
|       1010|         49.99|
|       1011|         49.99|
|       1004|         49.99|
|       1001|         49.99|
|       1013|         49.99|
|       1009|         49.99|
+-----------+--------------+



In [29]:
spark.sql("select customer_ID, max(amount) as maximum_amount from it009033_customers.customers_external where purchase_date>= '2023-05-01' and purchase_date <= '2023-06-30' group by 1 order by max(amount) desc limit 10").show()

+-----------+--------------+
|customer_ID|maximum_amount|
+-----------+--------------+
|       1015|         59.99|
|       1014|         54.99|
|       1010|         49.99|
|       1013|         49.99|
|       1009|         49.99|
|       1011|         49.99|
|       1005|         49.99|
|       1006|         49.99|
|       1001|         49.99|
|       1004|         49.99|
+-----------+--------------+



In [30]:
customers_df.show(2)

+-----------+-------------+----------+------+
|customer_ID|purchase_date|product_id|amount|
+-----------+-------------+----------+------+
|       1001|   2023-05-15|      1001| 49.99|
|       1002|   2023-05-16|      1002| 29.99|
+-----------+-------------+----------+------+
only showing top 2 rows



In [51]:
from pyspark.sql.functions import year, month
from pyspark.sql.functions import countDistinct
from pyspark.storagelevel import StorageLevel

In [32]:
df2 = customers_df.withColumn("p_year", year("purchase_date")).withColumn("p_month", month("purchase_date"))

df2.show()

+-----------+-------------+----------+------+------+-------+
|customer_ID|purchase_date|product_id|amount|p_year|p_month|
+-----------+-------------+----------+------+------+-------+
|       1001|   2023-05-15|      1001| 49.99|  2023|      5|
|       1002|   2023-05-16|      1002| 29.99|  2023|      5|
|       1003|   2023-05-17|      1003| 39.99|  2023|      5|
|       1004|   2023-05-18|      1004| 19.99|  2023|      5|
|       1005|   2023-05-19|      1005| 24.99|  2023|      5|
|       1001|   2023-05-20|      1002| 29.99|  2023|      5|
|       1002|   2023-05-21|      1003| 39.99|  2023|      5|
|       1003|   2023-05-22|      1004| 19.99|  2023|      5|
|       1004|   2023-05-23|      1005| 24.99|  2023|      5|
|       1005|   2023-05-24|      1001| 49.99|  2023|      5|
|       1001|   2023-05-25|      1003| 39.99|  2023|      5|
|       1002|   2023-05-26|      1004| 19.99|  2023|      5|
|       1003|   2023-05-27|      1005| 24.99|  2023|      5|
|       1004|   2023-05-

In [33]:
df3 = df2.groupBy("customer_ID", "p_year", "p_month").agg(countDistinct("p_month")).withColumnRenamed("count(DISTINCT p_month)", "distinct_months")
# df3 = df3.withColumnRenamed("count(DISTINCT p_month)", "distinct_months")
df3.show()

+-----------+------+-------+---------------+
|customer_ID|p_year|p_month|distinct_months|
+-----------+------+-------+---------------+
|       1011|  2023|      5|              1|
|       1002|  2023|      5|              1|
|       1004|  2023|      6|              1|
|       1014|  2023|      5|              1|
|       1004|  2023|      5|              1|
|       1009|  2023|      6|              1|
|       1013|  2023|      6|              1|
|       1002|  2023|      6|              1|
|       1013|  2023|      5|              1|
|       1001|  2023|      6|              1|
|       1007|  2023|      6|              1|
|       1007|  2023|      5|              1|
|       1015|  2023|      6|              1|
|       1006|  2023|      5|              1|
|       1005|  2023|      6|              1|
|       1006|  2023|      6|              1|
|       1014|  2023|      6|              1|
|       1003|  2023|      5|              1|
|       1010|  2023|      5|              1|
|       10

In [34]:
regular_custs = df3.filter("distinct_months == 1").groupBy("customer_ID").count()

In [35]:
regular_custs.show()

+-----------+-----+
|customer_ID|count|
+-----------+-----+
|       1010|    2|
|       1002|    2|
|       1012|    2|
|       1009|    2|
|       1013|    2|
|       1007|    2|
|       1011|    2|
|       1005|    2|
|       1001|    2|
|       1015|    2|
|       1014|    2|
|       1008|    2|
|       1004|    2|
|       1006|    2|
|       1003|    2|
+-----------+-----+



In [36]:
per_cust = customers_cached.persist(StorageLevel.MEMORY_AND_DISK_SER)

In [37]:
per_cust.count()

87498290

In [52]:
per_m_d = df2.groupBy("customer_ID", "p_year", "p_month").agg(countDistinct("p_month")).withColumnRenamed("count(DISTINCT p_month)", "distinct_months").persist(StorageLevel.MEMORY_AND_DISK_SER)
# df3 = df3.withColumnRenamed("count(DISTINCT p_month)", "distinct_months")
per_m_d.show()

+-----------+------+-------+---------------+
|customer_ID|p_year|p_month|distinct_months|
+-----------+------+-------+---------------+
|       1011|  2023|      5|              1|
|       1002|  2023|      5|              1|
|       1004|  2023|      6|              1|
|       1014|  2023|      5|              1|
|       1004|  2023|      5|              1|
|       1009|  2023|      6|              1|
|       1013|  2023|      6|              1|
|       1002|  2023|      6|              1|
|       1013|  2023|      5|              1|
|       1001|  2023|      6|              1|
|       1007|  2023|      6|              1|
|       1007|  2023|      5|              1|
|       1015|  2023|      6|              1|
|       1006|  2023|      5|              1|
|       1005|  2023|      6|              1|
|       1006|  2023|      6|              1|
|       1014|  2023|      6|              1|
|       1003|  2023|      5|              1|
|       1010|  2023|      5|              1|
|       10

In [40]:
per_m_d.show()

+-----------+------+-------+---------------+
|customer_ID|p_year|p_month|distinct_months|
+-----------+------+-------+---------------+
|       1011|  2023|      5|              1|
|       1002|  2023|      5|              1|
|       1004|  2023|      6|              1|
|       1014|  2023|      5|              1|
|       1004|  2023|      5|              1|
|       1009|  2023|      6|              1|
|       1013|  2023|      6|              1|
|       1002|  2023|      6|              1|
|       1013|  2023|      5|              1|
|       1001|  2023|      6|              1|
|       1007|  2023|      6|              1|
|       1007|  2023|      5|              1|
|       1015|  2023|      6|              1|
|       1006|  2023|      5|              1|
|       1005|  2023|      6|              1|
|       1006|  2023|      6|              1|
|       1014|  2023|      6|              1|
|       1003|  2023|      5|              1|
|       1010|  2023|      5|              1|
|       10

In [41]:
cached_m_d = df2.groupBy("customer_ID", "p_year", "p_month").agg(countDistinct("p_month")).withColumnRenamed("count(DISTINCT p_month)", "distinct_months").cache()
# df3 = df3.withColumnRenamed("count(DISTINCT p_month)", "distinct_months")
cached_m_d.show()

+-----------+------+-------+---------------+
|customer_ID|p_year|p_month|distinct_months|
+-----------+------+-------+---------------+
|       1011|  2023|      5|              1|
|       1002|  2023|      5|              1|
|       1004|  2023|      6|              1|
|       1014|  2023|      5|              1|
|       1004|  2023|      5|              1|
|       1009|  2023|      6|              1|
|       1013|  2023|      6|              1|
|       1002|  2023|      6|              1|
|       1013|  2023|      5|              1|
|       1001|  2023|      6|              1|
|       1007|  2023|      6|              1|
|       1007|  2023|      5|              1|
|       1015|  2023|      6|              1|
|       1006|  2023|      5|              1|
|       1005|  2023|      6|              1|
|       1006|  2023|      6|              1|
|       1014|  2023|      6|              1|
|       1003|  2023|      5|              1|
|       1010|  2023|      5|              1|
|       10

In [42]:
cached_m_d.show()

+-----------+------+-------+---------------+
|customer_ID|p_year|p_month|distinct_months|
+-----------+------+-------+---------------+
|       1011|  2023|      5|              1|
|       1002|  2023|      5|              1|
|       1004|  2023|      6|              1|
|       1014|  2023|      5|              1|
|       1004|  2023|      5|              1|
|       1009|  2023|      6|              1|
|       1013|  2023|      6|              1|
|       1002|  2023|      6|              1|
|       1013|  2023|      5|              1|
|       1001|  2023|      6|              1|
|       1007|  2023|      6|              1|
|       1007|  2023|      5|              1|
|       1015|  2023|      6|              1|
|       1006|  2023|      5|              1|
|       1005|  2023|      6|              1|
|       1006|  2023|      6|              1|
|       1014|  2023|      6|              1|
|       1003|  2023|      5|              1|
|       1010|  2023|      5|              1|
|       10

In [43]:
cached_m_d.unpersist()

customer_ID,p_year,p_month,distinct_months
1011,2023,5,1
1002,2023,5,1
1004,2023,6,1
1014,2023,5,1
1004,2023,5,1
1009,2023,6,1
1013,2023,6,1
1002,2023,6,1
1013,2023,5,1
1001,2023,6,1


In [44]:
MEMORY_ONLY = df2.groupBy("customer_ID", "p_year", "p_month").agg(countDistinct("p_month")).withColumnRenamed("count(DISTINCT p_month)", "distinct_months").persist(StorageLevel.MEMORY_ONLY)
# df3 = df3.withColumnRenamed("count(DISTINCT p_month)", "distinct_months")
MEMORY_ONLY.show()

+-----------+------+-------+---------------+
|customer_ID|p_year|p_month|distinct_months|
+-----------+------+-------+---------------+
|       1011|  2023|      5|              1|
|       1002|  2023|      5|              1|
|       1004|  2023|      6|              1|
|       1014|  2023|      5|              1|
|       1004|  2023|      5|              1|
|       1009|  2023|      6|              1|
|       1013|  2023|      6|              1|
|       1002|  2023|      6|              1|
|       1013|  2023|      5|              1|
|       1001|  2023|      6|              1|
|       1007|  2023|      6|              1|
|       1007|  2023|      5|              1|
|       1015|  2023|      6|              1|
|       1006|  2023|      5|              1|
|       1005|  2023|      6|              1|
|       1006|  2023|      6|              1|
|       1014|  2023|      6|              1|
|       1003|  2023|      5|              1|
|       1010|  2023|      5|              1|
|       10

In [45]:
regular_custs = MEMORY_ONLY.filter("distinct_months == 1").groupBy("customer_ID").count()
regular_custs.show()

+-----------+-----+
|customer_ID|count|
+-----------+-----+
|       1010|    2|
|       1002|    2|
|       1012|    2|
|       1009|    2|
|       1013|    2|
|       1007|    2|
|       1011|    2|
|       1005|    2|
|       1001|    2|
|       1015|    2|
|       1014|    2|
|       1008|    2|
|       1004|    2|
|       1006|    2|
|       1003|    2|
+-----------+-----+



In [46]:
regular_custs.show()

+-----------+-----+
|customer_ID|count|
+-----------+-----+
|       1010|    2|
|       1002|    2|
|       1012|    2|
|       1009|    2|
|       1013|    2|
|       1007|    2|
|       1011|    2|
|       1005|    2|
|       1001|    2|
|       1015|    2|
|       1014|    2|
|       1008|    2|
|       1004|    2|
|       1006|    2|
|       1003|    2|
+-----------+-----+



In [55]:
LEVEL = df2.groupBy("customer_ID", "p_year", "p_month").agg(countDistinct("p_month")).withColumnRenamed("count(DISTINCT p_month)", "distinct_months").persist(StorageLevel.MEMORY_ONLY_SER)
LEVEL = LEVEL.filter("distinct_months == 1").groupBy("customer_ID").count()
LEVEL.show()

+-----------+-----+
|customer_ID|count|
+-----------+-----+
|       1010|    2|
|       1002|    2|
|       1012|    2|
|       1009|    2|
|       1013|    2|
|       1007|    2|
|       1011|    2|
|       1005|    2|
|       1001|    2|
|       1015|    2|
|       1014|    2|
|       1008|    2|
|       1004|    2|
|       1006|    2|
|       1003|    2|
+-----------+-----+



In [56]:
LEVEL.show()

+-----------+-----+
|customer_ID|count|
+-----------+-----+
|       1010|    2|
|       1002|    2|
|       1012|    2|
|       1009|    2|
|       1013|    2|
|       1011|    2|
|       1007|    2|
|       1005|    2|
|       1001|    2|
|       1015|    2|
|       1014|    2|
|       1008|    2|
|       1004|    2|
|       1006|    2|
|       1003|    2|
+-----------+-----+



In [57]:
LEVEL = df2.groupBy("customer_ID", "p_year", "p_month").agg(countDistinct("p_month")).withColumnRenamed("count(DISTINCT p_month)", "distinct_months").persist(StorageLevel.MEMORY_AND_DISK)
LEVEL = LEVEL.filter("distinct_months == 1").groupBy("customer_ID").count()
LEVEL.show()

+-----------+-----+
|customer_ID|count|
+-----------+-----+
|       1010|    2|
|       1002|    2|
|       1012|    2|
|       1009|    2|
|       1013|    2|
|       1011|    2|
|       1007|    2|
|       1005|    2|
|       1001|    2|
|       1015|    2|
|       1014|    2|
|       1008|    2|
|       1004|    2|
|       1006|    2|
|       1003|    2|
+-----------+-----+



In [58]:
LEVEL.show()

+-----------+-----+
|customer_ID|count|
+-----------+-----+
|       1010|    2|
|       1002|    2|
|       1012|    2|
|       1009|    2|
|       1013|    2|
|       1007|    2|
|       1011|    2|
|       1005|    2|
|       1001|    2|
|       1015|    2|
|       1014|    2|
|       1008|    2|
|       1004|    2|
|       1006|    2|
|       1003|    2|
+-----------+-----+



In [59]:
LEVEL = df2.groupBy("customer_ID", "p_year", "p_month").agg(countDistinct("p_month")).withColumnRenamed("count(DISTINCT p_month)", "distinct_months").persist(StorageLevel.MEMORY_AND_DISK_SER)
LEVEL = LEVEL.filter("distinct_months == 1").groupBy("customer_ID").count()
LEVEL.show()

+-----------+-----+
|customer_ID|count|
+-----------+-----+
|       1010|    2|
|       1002|    2|
|       1012|    2|
|       1009|    2|
|       1013|    2|
|       1007|    2|
|       1011|    2|
|       1005|    2|
|       1001|    2|
|       1015|    2|
|       1014|    2|
|       1008|    2|
|       1004|    2|
|       1006|    2|
|       1003|    2|
+-----------+-----+



In [60]:
LEVEL.show()

+-----------+-----+
|customer_ID|count|
+-----------+-----+
|       1010|    2|
|       1002|    2|
|       1012|    2|
|       1009|    2|
|       1013|    2|
|       1007|    2|
|       1011|    2|
|       1005|    2|
|       1001|    2|
|       1015|    2|
|       1014|    2|
|       1008|    2|
|       1004|    2|
|       1006|    2|
|       1003|    2|
+-----------+-----+



In [61]:
LEVEL = df2.groupBy("customer_ID", "p_year", "p_month").agg(countDistinct("p_month")).withColumnRenamed("count(DISTINCT p_month)", "distinct_months").persist(StorageLevel.DISK_ONLY)
LEVEL = LEVEL.filter("distinct_months == 1").groupBy("customer_ID").count()
LEVEL.show()

+-----------+-----+
|customer_ID|count|
+-----------+-----+
|       1010|    2|
|       1002|    2|
|       1012|    2|
|       1009|    2|
|       1013|    2|
|       1011|    2|
|       1007|    2|
|       1005|    2|
|       1001|    2|
|       1015|    2|
|       1014|    2|
|       1008|    2|
|       1004|    2|
|       1006|    2|
|       1003|    2|
+-----------+-----+



In [62]:
LEVEL.show()

+-----------+-----+
|customer_ID|count|
+-----------+-----+
|       1010|    2|
|       1002|    2|
|       1012|    2|
|       1009|    2|
|       1013|    2|
|       1007|    2|
|       1011|    2|
|       1005|    2|
|       1001|    2|
|       1015|    2|
|       1014|    2|
|       1008|    2|
|       1004|    2|
|       1006|    2|
|       1003|    2|
+-----------+-----+



# B) The customer service team frequently needs to access the transaction history of a specific customer to resolve any issues or provide personalized assistance. Design a caching mechanism that allows fast retrieval of a customer's transaction history.(hint: you can use user defined functions to pass customer_id to get the transaction details)

In [63]:
customers_cached.show()

+-----------+-------------+----------+------+
|customer_ID|purchase_date|product_id|amount|
+-----------+-------------+----------+------+
|       1001|   2023-05-15|      1001| 49.99|
|       1002|   2023-05-16|      1002| 29.99|
|       1003|   2023-05-17|      1003| 39.99|
|       1004|   2023-05-18|      1004| 19.99|
|       1005|   2023-05-19|      1005| 24.99|
|       1001|   2023-05-20|      1002| 29.99|
|       1002|   2023-05-21|      1003| 39.99|
|       1003|   2023-05-22|      1004| 19.99|
|       1004|   2023-05-23|      1005| 24.99|
|       1005|   2023-05-24|      1001| 49.99|
|       1001|   2023-05-25|      1003| 39.99|
|       1002|   2023-05-26|      1004| 19.99|
|       1003|   2023-05-27|      1005| 24.99|
|       1004|   2023-05-28|      1001| 49.99|
|       1005|   2023-05-29|      1002| 29.99|
|       1001|   2023-05-30|      1003| 39.99|
|       1002|   2023-05-31|      1004| 19.99|
|       1003|   2023-06-01|      1005| 24.99|
|       1004|   2023-06-02|      1

In [78]:
def desc(ids):
    hist = customers_cached.select("purchase_date", "product_id", "amount").filter((customers_cached.customer_ID == ids)).cache()
    return hist

In [80]:
desc(1001).show()

+-------------+----------+------+
|purchase_date|product_id|amount|
+-------------+----------+------+
|   2023-05-15|      1001| 49.99|
|   2023-05-20|      1002| 29.99|
|   2023-05-25|      1003| 39.99|
|   2023-05-30|      1003| 39.99|
|   2023-06-04|      1003| 39.99|
|   2023-05-15|      1001| 49.99|
|   2023-05-20|      1002| 29.99|
|   2023-05-25|      1003| 39.99|
|   2023-05-30|      1003| 39.99|
|   2023-06-04|      1003| 39.99|
|   2023-05-15|      1001| 49.99|
|   2023-05-20|      1002| 29.99|
|   2023-05-25|      1003| 39.99|
|   2023-05-30|      1003| 39.99|
|   2023-06-04|      1003| 39.99|
|   2023-05-15|      1001| 49.99|
|   2023-05-20|      1002| 29.99|
|   2023-05-25|      1003| 39.99|
|   2023-05-30|      1003| 39.99|
|   2023-06-04|      1003| 39.99|
+-------------+----------+------+
only showing top 20 rows



In [50]:
def persisting(LEVELS):
    LEVEL = df2.groupBy("customer_ID", "p_year", "p_month").agg(countDistinct("p_month")).withColumnRenamed("count(DISTINCT p_month)", "distinct_months").persist(StorageLevel.LEVELS)
    LEVEL = LEVEL.filter("distinct_months == 1").groupBy("customer_ID").count()
    return LEVELS.show()

In [81]:
spark.sql("uncache table it009033_customers.customers_external")

""


In [82]:
spark.sql("clear cache")

""


In [83]:
spark.stop()